In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms, datasets, utils
from torch.autograd import Variable

torch.manual_seed(42)

# Step 1 Build data pipeline and import image dataset (split into training set and test set 5:5)

In [2]:
# data_dir is the address where you store your data data, I put it here under relative path
data_dir = "D:/zhuomian/2022_project/data"
split = 0.5
batch_size = 32
learning_rate = 0.001
num_epochs = 40

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)

device: cpu


In [3]:
# Build data pre-processing
data_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Resize((56,56)),
                                     transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# Import data into dataset
dataset = datasets.ImageFolder(root = data_dir, transform = data_transform)

# Now divide the data set into a training set and a test set
len_imgs = len(dataset.imgs)
train_set, test_set = torch.utils.data.random_split(dataset, [int(len_imgs*split), len_imgs-int(len_imgs*split)]) 

# Final construction of the data pipeline
train_loader = torch.utils.data.DataLoader(train_set,batch_size = batch_size,shuffle = True)
test_loader = torch.utils.data.DataLoader(test_set,batch_size = batch_size,shuffle = False)

# Step 2 Build the model Customized loss function and optimizer

In [4]:
class CNN_model(nn.Module):
    def __init__(self):
        super(CNN_model, self).__init__()
        # conv2d Parameters nn.Conv2d(in_channels=3, out_channels=32,kernel_size=5,stride=1,padding="same")
        # MaxPool2d Parameters nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv0 = nn.Sequential(         
            nn.Conv2d(32, 48, 1, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 48, 3, 1, padding = "same"),
            nn.BatchNorm2d(48),
            nn.ReLU(),
            nn.Conv2d(48, 32, 1, 1, padding = "same"),
            nn.BatchNorm2d(32),
            nn.ReLU(),
        )
        
        self.conv1 = nn.Sequential(         
            nn.Conv2d(3,  32, 3, 1, padding = "same"),                              
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)    
        )
        
        self.conv2 = nn.Sequential(         
            nn.Conv2d(32, 64, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        self.conv3 = nn.Sequential(         
            nn.Conv2d(64, 32, 3, 1, padding = "same"),     
            nn.ReLU(),                      
            nn.MaxPool2d(2, 2)                
        )
        
        # Fully connected layer with 21 output categories
        self.fc1 = nn.Sequential(nn.Linear(32 * 7 * 7, 128),nn.ReLU())
        self.fc2 = nn.Sequential(nn.Linear(128, 21))
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        identity = x
        x = self.conv0(x)
        x = identity + x
        # Flattening treatment x.view
        x = x.view(x.size(0), -1)       
        x = self.fc1(x)
        output = self.fc2(x)
        return output    # return x for visualization

In [5]:
model = CNN_model()
model.to(device)
# Choose whether to print the model or not, here it will not be printed, just comment
# print(model)

# Select loss function and optimizer
loss_func = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),
                       lr=learning_rate, betas=(0.9, 0.999),
                       eps=1e-07, amsgrad=False)

In [6]:
for para in model.conv1.parameters():
     #para.requires_grad = False
        print(para)

Parameter containing:
tensor([[[[-1.6705e-01,  9.4571e-02, -1.5766e-01],
          [ 2.4070e-02,  1.6082e-02,  6.7943e-02],
          [ 9.6198e-02, -1.0258e-01,  1.1997e-02]],

         [[-1.1331e-01,  1.3599e-01,  5.6600e-02],
          [ 1.6800e-01,  1.5191e-01,  8.6073e-02],
          [ 1.2170e-01,  1.8604e-04, -1.4379e-01]],

         [[-7.9157e-02, -7.5692e-02, -8.3891e-02],
          [-9.7344e-02, -1.8786e-01,  3.5846e-02],
          [-1.2211e-01, -1.6781e-01,  5.9788e-02]]],


        [[[-1.8027e-01,  4.1725e-02, -1.1213e-02],
          [ 1.0158e-01,  1.5943e-01,  9.4848e-02],
          [-1.5441e-01, -1.8395e-02,  8.7878e-02]],

         [[-1.5113e-01, -1.8605e-01, -1.7254e-01],
          [-1.4346e-01,  8.3391e-02, -5.7112e-02],
          [ 7.8017e-02, -1.0249e-01, -4.8690e-02]],

         [[-2.8694e-02,  1.8501e-01,  2.5433e-02],
          [-1.5733e-01,  1.4851e-01,  1.0754e-02],
          [ 1.7372e-01, -1.6462e-03, -1.5580e-01]]],


        [[[ 3.1883e-02, -1.0541e-01, -1.3230

# Step 3 Pass in the model to start training and testing

In [11]:
# Train the model and record the accuracy and loss values for each epoch during training
def train(num_epochs, model, loaders):
    model.train()   
    for para in model.conv0.parameters():
        para.requires_grad = False
    # The first stores the correct rate and the second stores the loss value
    Accy_list = []
    Loss_list = []
    
    total_step = len(loaders)    
    for epoch in range(num_epochs):
        if epoch == 30:
            for para in model.conv0.parameters():
                para.requires_grad = True
            for para in model.conv1.parameters():
                para.requires_grad = False
            for para in model.conv2.parameters():
                para.requires_grad = False
            for para in model.conv3.parameters():
                para.requires_grad = False    
            for para in model.fc1.parameters():
                para.requires_grad = False    
            for para in model.fc2.parameters():
                para.requires_grad = False    
        correct = 0
        L = 0
        for i, (images, labels) in enumerate(loaders):
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            loss = loss_func(outputs, labels)
            
            # Clear the previous gradient 
            optimizer.zero_grad()           
            
            # Backpropagate and update the parameters
            loss.backward()               
            optimizer.step()                
            
            _, predicted = torch.max(outputs.data, 1)
            correct += (predicted == labels).float().sum()
            L += loss.item()
            if (i+1) % 9 == 0:
                print ('Epoch: [{}/{}]\t|\tStep: [{}/{}]\t|\tLoss: {:.4f}' 
                       .format(epoch + 1, num_epochs, i + 1, total_step, loss.item()))
                
        accuracy = 100 * correct / len(train_set)
        print(accuracy.item())
        print("epoch: {:02d} | Accuracy: {:.2f} Loss: {:.4f}\n"
              .format(epoch + 1, accuracy, L))
        
        # Add the correct and lost values for this epoch to the corresponding list
        Accy_list.append(accuracy.item())
        Loss_list.append(L)
              
    return Accy_list, Loss_list


Accy_list, Loss_list = train(num_epochs, model, train_loader)

Epoch: [1/40]	|	Step: [9/27]	|	Loss: 0.0010
Epoch: [1/40]	|	Step: [18/27]	|	Loss: 0.0010
Epoch: [1/40]	|	Step: [27/27]	|	Loss: 0.0015
100.0
epoch: 01 | Accuracy: 100.00 Loss: 0.0282

Epoch: [2/40]	|	Step: [9/27]	|	Loss: 0.0011
Epoch: [2/40]	|	Step: [18/27]	|	Loss: 0.0007
Epoch: [2/40]	|	Step: [27/27]	|	Loss: 0.0020
100.0
epoch: 02 | Accuracy: 100.00 Loss: 0.0281

Epoch: [3/40]	|	Step: [9/27]	|	Loss: 0.0009
Epoch: [3/40]	|	Step: [18/27]	|	Loss: 0.0007
Epoch: [3/40]	|	Step: [27/27]	|	Loss: 0.0014
100.0
epoch: 03 | Accuracy: 100.00 Loss: 0.0262

Epoch: [4/40]	|	Step: [9/27]	|	Loss: 0.0010
Epoch: [4/40]	|	Step: [18/27]	|	Loss: 0.0006
Epoch: [4/40]	|	Step: [27/27]	|	Loss: 0.0010
100.0
epoch: 04 | Accuracy: 100.00 Loss: 0.0259

Epoch: [5/40]	|	Step: [9/27]	|	Loss: 0.0010
Epoch: [5/40]	|	Step: [18/27]	|	Loss: 0.0012
Epoch: [5/40]	|	Step: [27/27]	|	Loss: 0.0034
100.0
epoch: 05 | Accuracy: 100.00 Loss: 0.0286

Epoch: [6/40]	|	Step: [9/27]	|	Loss: 0.0009
Epoch: [6/40]	|	Step: [18/27]	|	Loss: 0.0

In [12]:
print(Accy_list)

[100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0]


In [13]:
print(Loss_list)

[0.028217472252435982, 0.02812801126856357, 0.02615501917898655, 0.025901138549670577, 0.02859144570538774, 0.02657598431687802, 0.024592135392595083, 0.02368088421644643, 0.025389270507730544, 0.023931028554216027, 0.02367784292437136, 0.025439877528697252, 0.0223713792511262, 0.021276581683196127, 0.022676451480947435, 0.023276998748769984, 0.02015635755378753, 0.02007583127124235, 0.01869113859720528, 0.01901692434330471, 0.029991569957928732, 0.0205937908613123, 0.0206129978178069, 0.02014805216458626, 0.0167252367537003, 0.018065183307044208, 0.01651908431085758, 0.0179175155935809, 0.015592358075082302, 0.01692393480334431, 0.01580591287347488, 0.01418165271752514, 0.012818979157600552, 0.011487347830552608, 0.011479640379548073, 0.010273979103658348, 0.009005244195577689, 0.009413074774784036, 0.008205056612496264, 0.008665166955324821]


In [14]:
# Test models
def test(model, loaders):
    model.eval()
    with torch.no_grad():
        correct = 0
        L = 0
        for images, labels in loaders:
            images = images.to(device)
            labels = labels.to(device)              
            outputs = model(images)
            # Get the loss value and add it up
            loss = loss_func(outputs, labels)
            L += loss.item()
            
             # Get the correct number of predicted samples
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).float().sum()
            
    accuracy = (100 * correct / len(test_set))
    print("Test data | Accuracy: {:.2f} %, Loss: {:.4f} ".format(accuracy, L))
    
    return accuracy, L

Test_acc, Test_loss = test(model, test_loader)

Test data | Accuracy: 86.21 %, Loss: 15.3560 
